# Merge all auctions into one

Combines several datasets and do some simple analyses.  


In [1]:
import sys
import os
import re
import json

In [2]:
with open('../assets/drz-settings-current.json', 'r') as fid:
    cfg = json.load(fid)
print(cfg['AUCTION'])

OPBOD = cfg['AUCTION']['kind'] == 'opbod'
AUCTION_ID = cfg['AUCTION']['id']
DATE = cfg['AUCTION']['date']
DATA_DIR = cfg['FILE_LOCATION']['data_dir']
#REGEX_DIR = cfg['FILE_LOCATION']['regex_dir']
auction_month = DATE[:4] + '-' + DATE[4:6]
if cfg['AUCTION']['kind'] == 'inschrijving':
    month_counter = re.sub('(-)(\d{2})', '\g<1>', AUCTION_ID)[5:8]
elif cfg['AUCTION']['kind'] == 'opbod':
    month_counter = re.sub('(-)(\d{2})(\d{2})', '-\g<2>', AUCTION_ID)[5:8]

#sys.path.insert(0, cfg['FILE_LOCATION']['code_dir'])

VERBOSE = int(cfg['GENERAL']['verbose'])
SAVE_METHOD = cfg['GENERAL']['save_method']


{'kind': 'opbod', 'id': '2025-0601', 'date': '20250607'}


In [3]:
if SAVE_METHOD == 'skip_when_exist':
    do_save = lambda fn: not(os.path.isfile(fn))
elif SAVE_METHOD == 'always_overwrite':
    do_save = lambda _: True
elif SAVE_METHOD == 'skip_save':
    do_save = lambda _: False
else:
    raise NotImplementedError(f'SAVE_METHOD: {SAVE_METHOD} not implemented')

In [4]:
TAG_SINGLE = "nbconvert_instruction:remove_single_output"

In [5]:
# dates of auction
# format: yyyy-mm; yyyy: year, mm: month (%Y-%m)
if OPBOD:
    AuctionDates = ['2019-11', '2019-12', 
                    '2020-01', '2020-02', '2020-03', '2021-09-09', '2021-10-10', '2021-11-11', '2021-12-12', 
                    '2022-01-01', '2022-02-02', '2022-04-04', '2022-05-05', '2022-06-06', '2022-08-08', '2022-09-09', '2022-10-10', '2022-11-11', '2022-12-12',
                    '2023-01-01', '2023-02-02', '2023-03-03', '2023-04-04', '2023-05-05', '2023-06-06', '2023-07-07', '2023-08-08', '2023-09-09', '2023-10-10', '2023-11-11', '2023-12-12',
                    '2024-01-01', '2024-02-02', '2024-03-03', '2024-04-04', '2024-05-05', '2024-06-06', '2024-07-07', '2024-08-08', '2024-09-09', '2024-10-10', '2024-11-11', '2024-12-12',
                    '2025-01-01', '2025-02-02'              , '2025-04-04', '2025-05-05', '2025-06-06',# '2025-07-07', '2025-08-08', '2025-09-09', '2025-10-10', '2025-11-11', '2025-12-12',
                   ]
else:    
    AuctionDates = ['2014-10', '2014-11', '2014-12', # old!
                    '2015-01', '2015-02', '2015-03', # old!
                    '2017-03', '2017-04', '2017-05', '2017-06', '2017-07', '2017-08', '2017-09', '2017-10', '2017-11', '2017-12', 
                    '2018-01', '2018-02', '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08', '2018-09', '2018-10', '2018-11', '2018-12',
                    '2019-01', '2019-02', '2019-03', '2019-04', '2019-05', '2019-06', '2019-07', '2019-08', '2019-09', '2019-10', '2019-11', '2019-12',
                    '2020-01', '2020-02', '2020-03', '2020-06', '2020-07', '2020-08', '2020-09', '2020-10', '2020-11', '2020-12',
                    '2021-01', '2021-02', '2021-03', '2021-04', '2021-05', 
                    '2021-06', '2021-06-16', '2021-07-07',  '2021-07-17', '2021-08-08', '2021-08-18', '2021-09-09', '2021-09-19', '2021-10-10', '2021-10-20', '2021-11-11', '2021-11-21', '2021-12-12', '2021-12-22',
                    '2022-01-01', '2022-01-21', '2022-02-02', '2022-02-22', '2022-03-03', '2022-03-23', '2022-04-04', '2022-04-24', '2022-05-05', '2022-05-25', '2022-06-06', 
                    '2022-07-07', '2022-07-27', '2022-08-08', '2022-08-28', '2022-09-09', '2022-09-29', '2022-10-10', '2022-10-30', '2022-11-11', '2022-11-31', '2022-12-12', '2022-12-32',
                    '2023-01-01', '2023-01-02', '2023-02-03', '2023-02-04', '2023-03-05', '2023-03-06', '2023-04-07', '2023-04-08', '2023-05-09', '2023-05-10', '2023-06-11', '2023-06-12',
                    '2023-07-13', '2023-07-14', '2023-08-15', '2023-08-16', '2023-09-17', '2023-10-19', '2023-10-20', '2023-11-21', '2023-11-22', '2023-12-23', '2023-12-24',
                    '2024-01-01', '2024-01-02', '2024-02-03', '2024-02-04', '2024-03-05', '2024-03-06', '2024-04-07', '2024-04-08', '2024-05-09', '2024-05-10', '2024-06-11', '2024-06-12',
                    '2024-07-13', '2024-07-14', '2024-08-15', '2024-08-16', '2024-09-17', '2024-09-18', '2024-10-19', '2024-10-20', '2024-11-21', '2024-11-22', '2024-12-23', '2024-12-24',
                    '2025-01-01', '2025-01-02', '2025-02-03', '2025-02-04', '2025-03-05', '2025-03-06', '2025-04-07', '2025-04-08', '2025-05-09', '2025-05-10', '2025-06-11', #'2025-06-12',
                    
                   ]  
if DATE[:6] not in [''.join(d.split('-')[:2]) for d in AuctionDates]:
    raise ValueError(f'{DATE} not in list that will be concatenated. Add to list.')

### Import modules 

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
plt.style.use('ggplot')

### Load reference tables

In [7]:
name_table = pd.read_excel(f"{cfg['FILE_LOCATION']['code_dir']}/assets/fieldname-lookuptable.ods", sheet_name='preferred_name', header=0, skipfooter=1, dtype=str)
name_table = name_table.dropna(how='all', axis=0).dropna(how='all', axis=1)
rename_dict={}
for i,r in name_table.iterrows():
    r=r.dropna()
    k = r.iloc[0]
    for option in r.iloc[1:]:
        rename_dict[option] = k

print(len(rename_dict))

545


In [8]:
# some have trailing _X-s as counter
# if option does not have counter use _1
pat = '(_X)+$'
for from_name, to_name in rename_dict.items():
    # count nr of _X
    lvl = [None,None]
    M = re.search(pat, from_name); 
    lvl[0] = int((M.end() - M.start())/2) if M is not None else 0
    M = re.search(pat, to_name); 
    lvl[1] = int((M.end() - M.start())/2) if M is not None else 0

    # if disbalance assume _1 
    # rename last _X to _1 repeat for diff in _X
    if lvl[0]<lvl[1]:
        for i in range(lvl[1]-lvl[0]):
            to_name = re.sub('_X(_1)?$', '_1\g<1>', to_name)
        rename_dict[from_name] = to_name

In [9]:
rename_dict_with_counter = {}
for k,v in rename_dict.items():
    pat = '_X(_\d)?$'
    if re.search(pat, v) is not None:
        # should have equal nr of _X
        n = [None,None]
        M = re.search(pat, k)
        n[0] = len(re.findall('_X', M[0]))
        M = re.search(pat, v)
        n[1] = len(re.findall('_X', M[0]))
        assert n[0] == n[1]
        rename_dict_with_counter[k] = v

for k in rename_dict_with_counter.keys():
    del rename_dict[k]

print(len(rename_dict), len(rename_dict_with_counter))

466 79


### Load all data

Auctions are saved monthly in a .pkl file

In [10]:
print('load data')
data = dict()
for AuctionDate in AuctionDates:
    # read data
    if OPBOD:
        fn = f'{DATA_DIR}/auctions/enriched-results/rdw-data-{AuctionDate}-opbod.pkl'

    else:    
        fn = f'{DATA_DIR}/auctions/enriched-results/rdw-data-{AuctionDate}.pkl'

    if VERBOSE > 0: print(fn)
    df = pd.read_pickle(fn)
    
    # flatten column levels
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = df.columns.map(lambda x: '_'.join(x))# if x[0] != 'drz' else '_'.join(x[1:]))
        
    # store in dict
    data[AuctionDate] = df
    
    
# When there are more auctions in a single month
if not OPBOD:
    # HACK ALERT!!
    data['2021-06-16'].index = data['2021-06-16'].index.map(lambda x: '{0}-06-{2}{1}'.format(*x.split('-')))
    data['2021-06'].index = data['2021-06'].index.map(lambda x: '{0}-06-{2}{1}'.format(*x.split('-')))
    
    # matrix of auction date
    auct_month = np.array([[int(k.split('-')[0]), int(k.split('-')[1])] for k in data.keys()])
    auct_month = np.unique(auct_month, axis=0)

    # 2021-07 there is a systematic index change
    for m in auct_month[((auct_month[:,0] == 2021) & (auct_month[:,1] >= 7)) | (auct_month[:,0] == 2022)]:
        for c in range(2):

            # multiplier
            #   2021: 10
            #   2022, 2023: 20
            if m[0] == 2021:
                multiplier = 10
            elif m[0] == 2022:
                multiplier = 20
            else:
                raise NotImplementedError

            # data key
            k = f'{m[0]}-{m[1]:02.0f}-{m[1]+(c*multiplier):02.0f}'
            if k not in data.keys():
                # skip if not exist
                continue

            # translation
            pat = '{0}-' + f'{m[1]:02.0f}' + '-{2}{1}'
            data[k].index = data[k].index.map(lambda x: pat.format(*x.split('-')))

            if VERBOSE > 0:
                print(f'{k} > {data[k].index[0]} .. {data[k].index[-1]}')
    # as of 2023 auctions are just incremented by 1
    for m in auct_month[(auct_month[:,0] >= 2023)]:
        for k in [k for k in data.keys() if k.startswith(f'{m[0]}-{m[1]:02.0f}')]:
            pat = '{0}-' + f'{m[1]:02.0f}' + '-{2}{1}'
            data[k].index = data[k].index.map(lambda x: pat.format(*x.split('-')))
            print(f'{k} > {data[k].index[0]} .. {data[k].index[-1]}')
            
            
        
if VERBOSE > 0:
    display({'text/plain': '\nlast rows of last file'}, raw=True, metadata={"tags":(TAG_SINGLE, )})
    display(data[AuctionDates[-1]].tail(), metadata={"tags":(TAG_SINGLE, )})

load data
/home/tom/bin/satdatsci/Saturday-Datascience/data/auctions/enriched-results/rdw-data-2019-11-opbod.pkl
/home/tom/bin/satdatsci/Saturday-Datascience/data/auctions/enriched-results/rdw-data-2019-12-opbod.pkl
/home/tom/bin/satdatsci/Saturday-Datascience/data/auctions/enriched-results/rdw-data-2020-01-opbod.pkl
/home/tom/bin/satdatsci/Saturday-Datascience/data/auctions/enriched-results/rdw-data-2020-02-opbod.pkl
/home/tom/bin/satdatsci/Saturday-Datascience/data/auctions/enriched-results/rdw-data-2020-03-opbod.pkl
/home/tom/bin/satdatsci/Saturday-Datascience/data/auctions/enriched-results/rdw-data-2021-09-09-opbod.pkl
/home/tom/bin/satdatsci/Saturday-Datascience/data/auctions/enriched-results/rdw-data-2021-10-10-opbod.pkl
/home/tom/bin/satdatsci/Saturday-Datascience/data/auctions/enriched-results/rdw-data-2021-11-11-opbod.pkl
/home/tom/bin/satdatsci/Saturday-Datascience/data/auctions/enriched-results/rdw-data-2021-12-12-opbod.pkl
/home/tom/bin/satdatsci/Saturday-Datascience/data/a

### Consolidate format
Older files are slightly different such as naming conventions etc. Here all results are consolidated into one format.

In [11]:
# Change type of early auctions for images
if VERBOSE > 0:
    print('Fields with images look like this:\n')
    df_ = pd.DataFrame(index=data.keys(), columns=['column', 'type', 'first row'])
    for k in data:
        im_cols = data[k].columns[
            data[k].columns.str.lower().isin(['image', 'images', 'drz__images'])
        ]
        example = data[k][im_cols[0]].dropna().values[0]
        df_.loc[k, 'type'] = str(type(example))
        df_.loc[k, 'column'] = im_cols[0]
        if isinstance(example, str):
            df_.loc[k, 'first row'] = example
        else:
            df_.loc[k, 'first row'] = len(example)
    with pd.option_context('display.max_row', 999):
        display(df_)

if OPBOD:
    # reformatting did not happen in IRS auctions
    pass
else:
    
    # convert string representation of list to real list
    # "['image1', 'image2']"
    for k in ['2017-03',
              '2017-04',
              '2017-05',
              '2017-06',
              '2017-07',
              '2017-08',
              '2017-09',
              '2017-10']:
        df_ = data[k].copy() # avoid performance warning
        df_['drz__Images'] = df_['Image'].apply(eval)
        data[k] = df_.drop(columns=['Image'])

    # convert string representation of list without quotes to real list
    # "[image1, image2]"
    for k in ['2017-11',
              '2017-12',
              '2018-01',
              '2018-02',
              '2018-03',
              '2018-04']:
        df_ = data[k].copy()
        df_['drz__Images'] = df_['Images'].apply(lambda s:eval(re.sub(r"^\[(.*)\]$",r"['\1']",re.sub(' *, *',"', '",s))))
        data[k] = df_.drop(columns=['Images'])
    

Fields with images look like this:



,column,type,first row
2019-11,Images,<class 'list'>,3
2019-12,Images,<class 'list'>,3
2020-01,Images,<class 'list'>,2
2020-02,Images,<class 'list'>,3
2020-03,Images,<class 'list'>,2
2021-09-09,Images,<class 'list'>,4
2021-10-10,Images,<class 'list'>,9
2021-11-11,Images,<class 'list'>,11
2021-12-12,Images,<class 'list'>,9
2022-01-01,Images,<class 'list'>,12


In [12]:
VERBOSE = 1
# Change dicttype of early auctions to regular fields
if VERBOSE > 0:
    print('rdwinfo look like this:\n')
    df_ = pd.DataFrame(index=data.keys(), columns=['column', 'type', 'first length'])
    for k in data:
        ri_cols = data[k].columns[
            data[k].columns.str.lower().isin(['rdwinfo'])
        ]
        if ri_cols.shape[0] == 0:
            continue
        example = data[k][ri_cols[0]].dropna().values[0]
        df_.loc[k, 'type'] = str(type(example))
        df_.loc[k, 'column'] = ri_cols[0]
        df_.loc[k, 'first length'] = len(example)
    with pd.option_context('display.max_row', 999):
        display(df_.dropna(how='all'))

if OPBOD:
    NotImplementedError
else:
    

    # Convert list to dict first
    # df_.groupby('type').get_group("<class 'list'>").index
    for k in ['2017-04',
              '2017-05',
              '2018-07',
              '2018-09',
              '2018-10']:
        df_ = data[k].copy()
        assert all(df_['rdwinfo'].apply(lambda x: len(x) if isinstance(x, list) else 0)<=1), 'Which item in list should I pick'
        df_['rdwinfo'] = df_['rdwinfo'].apply(lambda x: x[0] if isinstance(x, list) else None)
        data[k] = df_

    # expand dict to DF columns
    # df_.groupby('type').get_group("<class 'dict'>").index
    for k in [
        '2017-03', '2017-04', '2017-05', '2017-06', '2017-07', '2017-08',
        '2017-09', '2017-10', '2017-11', '2017-12', '2018-01', '2018-02',
        '2018-03', '2018-04', '2018-05', '2018-06', '2018-07', '2018-08',
        '2018-09', '2018-10', '2018-11', '2018-12', '2019-01', '2019-02',
        '2019-03'
    ]:
        df_ = data[k].copy()
        columns = set(df_['rdwinfo'].apply(lambda x: list(x.keys()) if x is not None else []).sum())

        df_ri = pd.json_normalize(df_['rdwinfo'], sep='|')
        df_ri.index = df_.index
        df_ri.columns = df_ri.columns.str.split('|', expand=True)
        columns_with_suffix = df_ri.loc[:, (slice(None), '1')].columns.get_level_values(0)
        columns_wo_suffix = np.setdiff1d(df_ri.columns.get_level_values(0), columns_with_suffix)

        assert df_ri.columns.nlevels == 2, 'more splits in columns'
        new_col_names = dict()
        for c in df_ri.columns:
            if c[0] in columns_wo_suffix:
                new_col_names[c] = c[0]
            elif c[0] in columns_with_suffix:
                new_col_names[c] = c[0] + '_' + str(int(c[1])+1)
        df_ri.columns = new_col_names.values()
        df_ri = df_ri.add_prefix('rdw_')
        # kenteken_1 and _2 in set. reduce to one
        cols = [c for c in df_ri.columns if c.startswith('rdw_kenteken_')]
        if len(cols) > 0:
            reg_plate = df_ri.loc[:, cols].bfill(axis=1).ffill(axis=1)
            assert (reg_plate.fillna('').nunique(axis=1) > 1).any() == False, 'regs not equal'
            df_ri['rdw_kenteken'] = reg_plate.rdw_kenteken_1
            df_ri.drop(columns=cols, inplace=True)
        existing_cols = np.intersect1d(df_ri.columns, df_.columns).tolist()
        existing_cols += ['rdwinfo']
        data[k] = pd.merge(df_.drop(columns=existing_cols), df_ri, left_index=True, right_index=True)


rdwinfo look like this:



,column,type,first length


In [13]:
# Other inconsistencies
if OPBOD:
    pass
else:
    # Other column names that changed in November 2017
    for k in ['2017-03',
              '2017-04',
              '2017-05',
              '2017-06',
              '2017-07',
              '2017-08',
              '2017-09',
              '2017-10']:
        data[k] = data[k].rename(columns={
            'draw':'drz__Draw',
            'nr':'drz__LotNr',
            'raw':'drz__Raw_text',
            'misc':'drz__SupInfo',
            'footnote':'drz__Note',
            'jfq':'drz__jfc',
        })
        

In [14]:
for auct_date, df_orig in data.items():

    df = df_orig.copy()
    # standardize column names
    df.rename(columns=rename_dict, inplace=True)
    # with counters
    rn_dict = {}
    for k,v in rename_dict_with_counter.items():
        search_pat = re.sub('(_X)+(_\d+)?$', '_', k)
        sel_rn = df.columns.str.startswith(search_pat)
        if any(sel_rn):
            for col in df.columns[sel_rn]:
                if ('type1' in v) and ('type_1' in k):
                    counter = re.search('(_type_1)?((_\d+)+)$', col)[2] # ignore if ..type_1.. is in name
                else:
                    counter = re.search('(_\d+)+$', col)[0]
                newv = re.sub('(_X)+(_\d+)*$', counter + '\g<2>', v)
                rn_dict[col] = newv
                
                
    df.rename(columns=rn_dict, inplace=True)
    # this introduces duplicates in column names

    # drop second, keep first
    vc = df.columns.value_counts()
    duplicate_cols = vc[vc>1].index
    for c in duplicate_cols:
        # get column number
        idx = np.where(df.columns.get_loc(c))[0]
        assert len(idx) == 2 # for now only do first/second
        # update first col
        df.iloc[:,idx[0]] = df.iloc[:,idx[0]].combine_first(df.iloc[:,idx[1]])
        # drop second col
        df = df.iloc[:, [j for j in range(len(df.columns)) if j != idx[1]]]

    data[auct_date] = df
    del df_orig

/tmp/ipykernel_365232/921391297.py:32: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '2022-02-1000           NaN
2022-02-1002           NaN
2022-02-1003           NaN
2022-02-1004    20190726.0
2022-02-1005           NaN
2022-02-1006           NaN
2022-02-1007           NaN
2022-02-1010           NaN
2022-02-1013    20190923.0
2022-02-1014           NaN
2022-02-1015           NaN
2022-02-1017    20190312.0
2022-02-1018    20191205.0
2022-02-1020    20191217.0
2022-02-1021           NaN
2022-02-1022    20191203.0
2022-02-1023    20191017.0
2022-02-1025    20190507.0
2022-02-1026    20190822.0
2022-02-1027           NaN
2022-02-1028           NaN
2022-02-1029           NaN
2022-02-1030           NaN
2022-02-1031           NaN
2022-02-1032           NaN
2022-02-1033           NaN
2022-02-1034           NaN
2022-02-1036           NaN
2022-02-1037           NaN
2022-02-1038    20190704.0
2022-02-1039           NaN
2022-0

# Merge all auctions into one data frame

In [15]:
all_data = pd.concat(data)
df = all_data.droplevel(0)

assert df.index.is_unique, 'Lot name is not unique.'

In [16]:
def get_maxmin_suffix(index):

    pat = r'((.+)(_\d+))?$'
    
    split = np.array([[f] + list(re.findall(pat,f)[0]) for f in index])

    has_counter = [f != '' for f in split[:,2]]
    has_no_counter = [not(s) for s in has_counter]
    
    subsel = list(split[has_no_counter, 0])
        
    for col in np.unique(split[has_counter,2]):
        assert col != ''
        sel = split[:,2] == col
        if any([f=='' for f in split[sel,-1]]):
            subsel += [str(col)]
            continue
        num = [int(re.sub('^_', '', i)) for i in split[sel,-1]]
        i_max = np.argmax(num)
        i_min = np.argmin(num)
        subsel += [str(split[sel,0][i_min]), str(split[sel,0][i_max])]
    return subsel

if VERBOSE > 0:
    # Show columns added
    col_exist = pd.DataFrame.from_dict({k:all_data.columns.isin(d.columns) for k,d in data.items()}, orient='index', columns=all_data.columns)
    # deprecated: col_filled = all_data.isna().all(level=0, axis=0) == False
    col_filled = all_data.notna().groupby(level=0).any()

    idx = col_exist.apply(lambda x:np.where(x)[0])

    first_occurence = col_exist.index[idx.apply(lambda x: x[0])]
    last_occurence = col_exist.index[idx.apply(lambda x: x[-1])]
    nr_occurence = idx.apply(len)
    pct_occurence = idx.apply(lambda x: 100*(len(x)/(col_exist.shape[0]-x[0])))

    idx = col_filled.apply(lambda x:np.where(x)[0])
    last_filled = col_filled.index[idx.apply(lambda x: x[-1] if len(x)>0 else col_exist.shape[0]-1)]
    nr_filled = idx.apply(len)
    pct_filled = idx.apply(lambda x: 100*(len(x)/(col_exist.shape[0]-x[0])) if len(x)>0 else 0)


    df_ = pd.DataFrame(zip(first_occurence, last_occurence, nr_occurence, last_filled, nr_filled, pct_occurence, pct_filled), 
                       index=col_exist.columns, 
                       columns=['first', 'last', 'n', 'last_filled', 'n_not_empty', 'pct_since_first', 'pct_filled'])



    for _by_source, df__ in df_.groupby(df_.index.str.split('_').str.get(0)):
        display({'text/html':f'<h1>{_by_source}</h1>'}, raw=True)
        for g,df___ in df__.groupby('first'):          
            if g == col_exist.index[0]:
                # first auction
                print(f'First data in {g}: {df___.shape[0]:.0f}')
                continue
            print(f'Fields added in {g} n={df___.shape[0]}', end= ' ')
            subsel_cols = get_maxmin_suffix(df___.index)
            with pd.option_context("display.max_rows", None):
                display(df___.loc[subsel_cols,:])#.sort_index())#.sort_values(by='last_filled', ascending=False))

drz

First data in 2019-11: 63
Fields added in 2021-09-09 n=5 

,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
drz__lot_counter,2021-09-09,2025-06-06,43,2025-06-06,43,100.000000,100.000000
drz__disclaim5,2021-09-09,2025-06-06,43,2025-06-06,43,100.000000,100.000000
drz__disclaim6,2021-09-09,2025-06-06,43,2025-06-06,43,100.000000,100.000000
drz__no_nlreg19,2021-09-09,2025-06-06,43,2025-06-06,43,100.000000,100.000000
drz__wok,2021-09-09,2025-06-06,42,2025-06-06,42,97.674419,97.674419


Fields added in 2022-01-01 n=3 

,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
drz__first_reg_abroad,2022-01-01,2025-06-06,39,2025-06-06,39,100.0,100.0
drz__no_nap,2022-01-01,2025-06-06,39,2025-06-06,39,100.0,100.0
drz__key_fixed,2022-01-01,2025-06-06,39,2025-06-06,39,100.0,100.0


...

,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
rdw_handelsbenaming_fabrikant_typeaanduidingfabrikant_4,2025-04-04,2025-06-06,3,2025-06-06,3,100.000000,100.000000
rdw_handelsbenaming_fabrikant_typeaanduidingfabrikant_5,2025-04-04,2025-06-06,3,2025-06-06,3,100.000000,100.000000
rdw_merk_uitvoering_composite_key_7,2025-04-04,2025-04-04,1,2025-04-04,1,33.333333,33.333333
rdw_merk_uitvoering_composite_key_7,2025-04-04,2025-04-04,1,2025-04-04,1,33.333333,33.333333
rdw_merk_uitvoering_merkcoderdw_7,2025-04-04,2025-04-04,1,2025-04-04,1,33.333333,33.333333
rdw_merk_uitvoering_merkcoderdw_7,2025-04-04,2025-04-04,1,2025-04-04,1,33.333333,33.333333
rdw_merk_uitvoering_volgnummermerk_7,2025-04-04,2025-04-04,1,2025-04-04,1,33.333333,33.333333
rdw_merk_uitvoering_volgnummermerk_7,2025-04-04,2025-04-04,1,2025-04-04,1,33.333333,33.333333


Fields added in 2025-05-05 n=44 

,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
rdw_geconstateerde_gebreken_met_beschrijving_aantal_gebreken_geconstateerd_6_10,2025-05-05,2025-05-05,1,2025-05-05,1,50.0,50.0
rdw_geconstateerde_gebreken_met_beschrijving_aantal_gebreken_geconstateerd_6_12,2025-05-05,2025-05-05,1,2025-05-05,1,50.0,50.0
rdw_geconstateerde_gebreken_met_beschrijving_einddatum_gebrek_6_10,2025-05-05,2025-05-05,1,2025-06-06,0,50.0,0.0
rdw_geconstateerde_gebreken_met_beschrijving_einddatum_gebrek_6_12,2025-05-05,2025-05-05,1,2025-06-06,0,50.0,0.0
rdw_geconstateerde_gebreken_met_beschrijving_einddatum_gebrek_dt_6_10,2025-05-05,2025-05-05,1,2025-06-06,0,50.0,0.0
rdw_geconstateerde_gebreken_met_beschrijving_einddatum_gebrek_dt_6_12,2025-05-05,2025-05-05,1,2025-06-06,0,50.0,0.0
rdw_geconstateerde_gebreken_met_beschrijving_gebrek_artikel_nummer_6_10,2025-05-05,2025-05-05,1,2025-05-05,1,50.0,50.0
rdw_geconstateerde_gebreken_met_beschrijving_gebrek_artikel_nummer_6_12,2025-05-05,2025-05-05,1,2025-05-05,1,50.0,50.0
rdw_geconstateerde_gebreken_met_beschrijving_gebrek_identificatie_6_10,2025-05-05,2025-05-05,1,2025-05-05,1,50.0,50.0
rdw_geconstateerde_gebreken_met_beschrijving_gebrek_identificatie_6_12,2025-05-05,2025-05-05,1,2025-05-05,1,50.0,50.0


Fields added in 2025-06-06 n=275 

,first,last,n,last_filled,n_not_empty,pct_since_first,pct_filled
rdw_geconstateerde_gebreken_met_beschrijving_aantal_gebreken_geconstateerd_2_17,2025-06-06,2025-06-06,1,2025-06-06,1,100.0,100.0
rdw_geconstateerde_gebreken_met_beschrijving_aantal_gebreken_geconstateerd_2_17,2025-06-06,2025-06-06,1,2025-06-06,1,100.0,100.0
rdw_geconstateerde_gebreken_met_beschrijving_einddatum_gebrek_2_17,2025-06-06,2025-06-06,1,2025-06-06,0,100.0,0.0
rdw_geconstateerde_gebreken_met_beschrijving_einddatum_gebrek_2_17,2025-06-06,2025-06-06,1,2025-06-06,0,100.0,0.0
rdw_geconstateerde_gebreken_met_beschrijving_einddatum_gebrek_dt_2_17,2025-06-06,2025-06-06,1,2025-06-06,0,100.0,0.0
rdw_geconstateerde_gebreken_met_beschrijving_einddatum_gebrek_dt_2_17,2025-06-06,2025-06-06,1,2025-06-06,0,100.0,0.0
rdw_geconstateerde_gebreken_met_beschrijving_gebrek_artikel_nummer_2_17,2025-06-06,2025-06-06,1,2025-06-06,1,100.0,100.0
rdw_geconstateerde_gebreken_met_beschrijving_gebrek_artikel_nummer_2_17,2025-06-06,2025-06-06,1,2025-06-06,1,100.0,100.0
rdw_geconstateerde_gebreken_met_beschrijving_gebrek_identificatie_2_17,2025-06-06,2025-06-06,1,2025-06-06,1,100.0,100.0
rdw_geconstateerde_gebreken_met_beschrijving_gebrek_identificatie_2_17,2025-06-06,2025-06-06,1,2025-06-06,1,100.0,100.0


- - - -
# Continue with cars only

In [17]:
# select data
car=df[df.drz__LotType == 'Personenauto'].copy()

if VERBOSE > 0:
    # print lots that are cars per auction
    df_ = pd.DataFrame(columns=['year', 'month', 'lot'])
    df_.year = pd.Series(car.index).apply(lambda x: int(x.split('-')[0]))
    df_.month = pd.Series(car.index).apply(lambda x: int(x.split('-')[1]))
    df_.lot = pd.Series(car.index).apply(lambda x: int(x.split('-')[2]))
    df_['sfx'] = df_.lot.apply(lambda x: 0 if x<9999 else int(str(x)[-2:]))
    with pd.option_context('display.max_rows', 999):
        display(df_.groupby(['year', 'month']).agg({'lot': ['min', 'max'], 'sfx': ['min', 'max', 'count']}).sort_index())
    

display(car.tail(), metadata={"tags":(TAG_SINGLE, )})

lot       sfx          
             min   max min max count
year month                          
2019 11     1005  8334   0   0   208
     12     1001  9429   0   0   170
2020 1      1004  7312   0   0   174
     2      1004  8304   0   0    35
     3      1007  1913   0   0   139
2021 9      1000  1119   0   0    87
     10     1000  1013   0   0    10
     11     1002  1060   0   0    36
     12     1000  1055   0   0    42
2022 1      1000  1060   0   0    38
     2      1002  1052   0   0    30
     4      1001  1022   0   0    10
     5      1004  1015   0   0     7
     6      1001  1019   0   0    13
     8      1001  1001   0   0     1
     9      1003  1012   0   0     9
     10     1000  1008   0   0     4
     11     1002  1063   0   0    35
     12     1001  1163   0   0    56
2023 1      1001  1076   0   0    44
     2      1009  1078   0   0    18
     3      1002  1098   0   0    36
     4      1031  1190   0   0    50
     5      1004  1207   0   0    73
     6      1002  1282   0   0   108
     7      1003  1177   0   0    57
     8      1003  1318   0   0   126
     9      1008  1172   0   0    79
     10     1007  1179   0   0    51
     11     1006  1280   0   0    99
     12     1037  1299   0   0    73
2024 1      1002  1265   0   0   121
     2      1007  1136   0   0    43
     3      1009  1231   0   0    80
     4      1002  1377   0   0   137
     5      1002  1323   0   0    97
     6      1001  1314   0   0    89
     7      1005  1309   0   0    95
     8      1010  1469   0   0   142
     9      1002  1256   0   0    94
     10     1007  1217   0   0    86
     11     1008  1276   0   0   112
     12     1001  1324   0   0   103
2025 1      1000  1371   0   0    97
     2      1004  1183   0   0    68
     4      1004  1274   0   0    98
     5      1007  1230   0   0    83
     6      1005  1251   0   0    73

# Save file

In [18]:
file_name = rf'{DATA_DIR}/cars-from-all-auctions.pkl'
if OPBOD:
    file_name = file_name.replace('.pkl', '-opbod.pkl')
    
if True or do_save(file_name): # always overwrite merged data file
    print(file_name)
    car.to_pickle(file_name)
else:
    print(f'Skip. {file_name} exists or saving is disabled in settings.')
    


/home/tom/bin/satdatsci/Saturday-Datascience/data/cars-from-all-auctions-opbod.pkl


# Write example results to file
based on: https://stackoverflow.com/a/33869154

In [19]:
def pandas_df_to_markdown_table(df):
    fmt = ['-----' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    return df_formatted.to_csv(sep="|", index=False)


In [20]:
file_name = f"{cfg['FILE_LOCATION']['code_dir']}/assets/example-table-of-cars.md"

example = car.tail(10).copy()

# trim some long fields
example.rdwinfo = '.. rdw info ..'
example.Raw_text = '.. raw text ..'
example.SupInfo = '.. suplm. info. ..'

# convert to md
try:
    table_text = example.to_markdown()
except ImportError:
    print('Fallback')
    table_text = pandas_df_to_markdown_table(example)

# save
if do_save(file_name):
    with open(file_name,'w') as file:
        file.write(table_text)

    print('A markdown table is available as\n\t{}'.format(file_name))
else:
    print(f'Skip. {file_name} exists or saving is disabled in settings.')
    display(example, metadata={"tags":(TAG_SINGLE, )})
    


Skip. /home/tom/bin/satdatsci/Saturday-Datascience/code/assets/example-table-of-cars.md exists or saving is disabled in settings.


In [21]:
# idx = df.index.str.split('-', expand=True).droplevel(-1)
# is_last_auction = idx == idx[-1]
# subsel_reg = df.loc[is_last_auction, 'Reg'].drop_duplicates().values
# VERBOSE

In [22]:
# show recurring lots (based on registration no.)

vc = df.drz__Reg.value_counts()

# drop obvious
vc.drop(['onbekend', ''], inplace=True, errors='ignore')

# reduce output
if VERBOSE < 2:
    # only of last auction
    idx = df.index.str.split('-', expand=True).droplevel(-1)
    is_last_auction = idx == idx[-1]
    subsel_reg = df.loc[is_last_auction, 'drz__Reg'].drop_duplicates().values
    vc = vc[vc.index.isin(subsel_reg)]
    
# show recurring
for r in vc[vc > 1].index:
    # title
    print(r, vc[r], end=': ')
    sel_reg = df.drz__Reg==r
    print(' - '.join(list(df.loc[sel_reg, [
        'drz__ItemBrand', 'drz__ItemType', 'rdw_gekentekende_voertuigen_datum_eerste_tenaamstelling_in_nederland'
    ]].drop_duplicates(ignore_index=True).iloc[0].astype(str))), end=' ')
    
    df_ = df.loc[sel_reg, list(df.columns.drop([
        'drz__Raw_text', 'rdwinfo', 'rdw_merk_registratie_datum', 'rdw_merk_registratie_datum_dt', 'rdw_merkcode', 'drz__Images', 'drz__Source'
    ], errors='ignore'))].sort_values(by='drz__Reg').T

    # convert index to nr of days since first
    ix = pd.Series(df_.columns.map(lambda x: pd.to_datetime('/'.join(x.split('-')[:2])))).diff().dt.days.copy()
    ix[0] = 0
    ix=ix.astype(int).astype(str)
    ix[0] = pd.to_datetime('/'.join(df_.columns[0].split('-')[:2])).strftime('%Y/%m')

    df_.columns=ix

    if (VERBOSE < 2) | (df_.shape[1] > 4):

        vc2 = df_.map(lambda x: str(x) if isinstance(x, list) else x).nunique(axis=1)
        print('')
        with pd.option_context('display.max_rows', 999):
            cols = list(vc2[vc2>1].index)
            cols += ['drz__OdoKM']
            cols += ['drz__OdoMLS']
            display(df_.loc[cols,:])
    elif df_.shape[1] > 2:
        display(df_.loc[['drz__Price'],:].T)
    else:
        print(*list(df_.columns))
        

06-ZRK-3 4: PEUGEOT - 508 - 20130111.0 


,2024/12,31,90,61
drz__Title,Kavel K2412011093,Kavel K2501011292,Kavel K2504011230,Kavel K2506011081
drz__Price,0.0,0.0,0.0,1050.0
drz__N_images,22,23,25,25
drz__Note,Deze kavel is op 28-11-2024 komen te vervallen.,Deze kavel is op 06-01-2025 komen te vervallen.,False,False
drz__LotNr,K2412011093 is vervallen,K2501011292 is vervallen,K2504011230,K2506011081
drz__OdoKM,,,568.622,568.622
drz__NAP,,,logisch,logisch
drz__Mfdate,,,11.01.2013,11.01.2013
drz__APKdate,,,23.09.2025,23.09.2025
drz__taxi,False,False,True,True


7-VZV-72 4: MERCEDES-BENZ - sprinter - 20130506.0 


,2024/03,61,61,335
drz__Title,Kavel K2403011157,Kavel K2405011047,Kavel K2407011037,Kavel K2506011026
drz__Price,0.0,0.0,0.0,2800.0
drz__Note,False,Deze kavel is op 07-05-2024 komen te vervallen.,Deze kavel is op 02.07.2024 komen te vervallen.,False
drz__LotNr,K2403011157,K2405011047 is vervallen,K2407011037 is vervallen,K2506011026
drz__OdoKM,470.425,,,470.425
drz__NAP,onlogisch,,,onlogisch
drz__Mfdate,09.02.2011,,,09.02.2011
drz__diesel,True,False,False,True
drz__SupInfo,\nRechter schuifdeur zit vast.,NaN,NaN,\nRechter schuifdeur zit vast.\nBij deze veili...
rdw_assen_aantal_assen_1,2,2,2,2.0


VB-307-S 3: MERCEDES-BENZ - citan - 20130223.0 


...

,2024/06,365
drz__Title,Kavel K2406011283,Kavel K2506011178
drz__Price,0.0,950.0
drz__Note,Deze kavel is op 03-06-2024 komen te vervallen.,False
drz__LotNr,K2406011283 is vervallen,K2506011178
drz__OdoKM,,324.477
drz__NAP,,logisch
drz__Mfdate,,02.01.2008
drz__diesel,False,True
rdw_assen_aantal_assen_1,2,2.0
rdw_assen_aantal_assen_2,2,2.0


5-KKX-11 2: VOLKSWAGEN - golf - 20130523.0 


,2025/01,151
drz__Title,Kavel K2501011367,Kavel K2506011066
drz__Price,5200.0,0.0
drz__N_images,19,20
drz__Note,False,Deze kavel is op 28-05-2025 komen te vervallen.
drz__LotNr,K2501011367,K2506011066 is vervallen
drz__OdoKM,313.387,
drz__NAP,logisch,
drz__Mfdate,23.05.2013,
drz__APKdate,01.11.2025,
drz__benzine,True,False


GH-747-X 2: AUDI - a3 sportback e-tron - 20150530.0 


,2025/04,61
drz__Title,Kavel K2504011127,Kavel K2506011090
drz__Price,0.0,6800.0
drz__Note,Deze kavel is op 09-04-2024 komen te vervallen.,False
drz__LotNr,K2504011127 is vervallen,K2506011090
drz__OdoKM,,289.831
drz__NAP,,logisch
drz__Mfdate,,30.05.2015
drz__hybrid,False,True
drz__automatic,False,True
rdw_gekentekende_voertuigen_TimeStamp,2025-04-12 14:23:39,2025-06-06 19:08:13
